In [1]:
from requests_html import HTMLSession, HTML
from selenium import webdriver
import time
import re

In [2]:
#Kiểm tra xem dữ liệu có hợp pháp không
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://www.worldweatheronline.com/robots.txt')
rp.read()
rp.can_fetch('*', 'https://www.worldweatheronline.com/ho-chi-minh-city-weather-history/vn.aspx')

True

In [3]:
session = HTMLSession()

In [24]:
browser = webdriver.Chrome(executable_path='chromedriver.exe')

In [5]:
url = "https://www.worldweatheronline.com/ho-chi-minh-city-weather-history/vn.aspx"

In [17]:
#danh sách các năm lấy dữ liệu
year = [2017,2018,2019]

In [7]:
#Tên file chứa data
filename='weather_data.csv'

In [8]:
#tập hợp các loại thời tiết (để phân loại và chuyển về 1 số loại nhất định)
list_w = set()

#mở file
with open(filename,'w') as f:
        f.write('time\tmonth\ttemperature\tfeelslike\twind\tdirection\tgust\tcloud\thumidity\tprecipitation\tpressure\tweather\n')

In [23]:
def parse_data (url,list_year,filename):
    browser.get(url)
    for y in year:
        if ((y%4 == 0)|(y%400==0)):
            ferb_day=30
        else:
            ferb_day = 29
        #danh sách ngày trong tháng (để tìm kiếm ngày trong web)
        #thêm 0 để chỉ số bằng số tháng
        day = [[0],
                  range(1,32),
                  range(1,ferb_day),
                  range(1,32),
                  range(1,31),
                  range(1,32),
                  range(1,31),
                  range(1,32),
                  range(1,32),
                  range(1,31),
                  range(1,32),
                  range(1,31),
                  range(1,32)]    
        for m in range(12):
            m +=1
            if ((m==12)&(y==2019)):
                break;#Chỉ lấy đến tháng 11 năm 2019
            for d in day[m]:
                if (d<10):
                    d = f"0{d}"
                if (m<10):
                    m_str = f"0{m}"
                    text = f"{m_str}{d}{y}"
                else:
                    text = f"{m}{d}{y}"
                #Xác định vị trí của textbox và điền giá trị vào
                textbox = browser.find_element_by_xpath("//input[@id='ctl00_MainContentHolder_txtPastDate']")
                textbox.click()
                print(text)
                textbox.send_keys(text)
                #Bấm nút để lấy dữ liệu
                button = browser.find_element_by_xpath("//input[@id='ctl00_MainContentHolder_butShowPastWeather']")
                button.click()
                time.sleep(1)#Nghỉ 1 giây để trang có thời gian load
                
                r = HTML(html=browser.page_source)
                
                #kiểm tra xem page đã load đúng chưa
                #nếu sai thì tăng thời gian sleep
                print(r.find(".block_title",first=True).text)
                
                #lấy dữ liệu
                
                row = r.find(".tb_date",first=True)
                times =re.findall("\d{2}:\d{2}", row.text)
                
                row = r.find(".tb_temp",first=True)
                temp  = list(map(float,re.findall("(\d{1,2})",row.text)))
                
                row = r.find(".tb_feels",first=True)
                feels = list(map(float,re.findall("(\d{1,2})",row.text)))
                
                row = r.find(".tb_wind",first=True).text
                wind = list(map(float,re.findall("(\d{1,2}) km/h",row)))
                direction = re.findall(r"\n([A-Z]{1,3})",row)
                
                row = r.find(".tb_gust",first=True).text
                gust = list(map(float,re.findall("\d{1,2}",row)))
                
                row = r.find(".tb_cloud",first=True).text
                cloud = list(map(float,re.findall("\d{1,2}",row)))
                
                row = r.find(".tb_humidity",first=True).text
                humidity = list(map(float,re.findall("\d{1,2}",row)))
                
                row = r.find(".tb_precip",first=True).text
                precip = list(map(float,re.findall("(\d{1}.\d{1,3})",row)))
                
                row = r.find(".tb_pressure",first=True).text
                pressure = list(map(float,re.findall("\d{1,4}",row)))
                
                weather = []
                row = r.find(".tb_weather",first=True)
                att = row.find(".tb_item_img")
                for a in att:
                    weather += [a.attrs['alt']]
                    list_w.add(a.attrs['alt'])
                
                #ghi dữ liệu ra file csv
                with open(filename,"a") as f:
                    for i in range(8):
                        f.write(f"{times[i]}\t{m}\t{temp[i]}\t{feels[i]}\t{wind[i]}\t{direction[i]}\t{gust[i]}\t{cloud[i]}\t{humidity[i]}\t{precip[i]}\t{pressure[i]}\t{weather[i]}\n")
                

In [25]:
parse_data(url,year,filename)

10012019
Tue 01st Oct, 2019
10022019
Wed 02nd Oct, 2019
10032019
Thu 03rd Oct, 2019
10042019
Fri 04th Oct, 2019
10052019
Sat 05th Oct, 2019
10062019
Sun 06th Oct, 2019
10072019
Mon 07th Oct, 2019
10082019
Tue 08th Oct, 2019
10092019
Wed 09th Oct, 2019
10102019
Thu 10th Oct, 2019
10112019
Fri 11th Oct, 2019
10122019
Sat 12th Oct, 2019
10132019
Sun 13th Oct, 2019
10142019
Mon 14th Oct, 2019
10152019
Tue 15th Oct, 2019
10162019
Wed 16th Oct, 2019
10172019
Thu 17th Oct, 2019
10182019
Fri 18th Oct, 2019
10192019
Sat 19th Oct, 2019
10202019
Sun 20th Oct, 2019
10212019
Mon 21st Oct, 2019
10222019
Tue 22nd Oct, 2019
10232019
Wed 23rd Oct, 2019
10242019
Thu 24th Oct, 2019
10252019
Fri 25th Oct, 2019
10262019
Sat 26th Oct, 2019
10272019
Sun 27th Oct, 2019
10282019
Mon 28th Oct, 2019
10292019
Tue 29th Oct, 2019
10302019
Wed 30th Oct, 2019
10312019
Thu 31st Oct, 2019
11012019
Fri 01st Nov, 2019
11022019
Sat 02nd Nov, 2019
11032019
Sun 03rd Nov, 2019
11042019
Mon 04th Nov, 2019
11052019
Tue 05th No